In [3]:
#pip install opencv-python 


Note: you may need to restart the kernel to use updated packages.


# Data Preprocessing

In [2]:
import cv2,os
data_path='./Dataset/'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary
print(label_dict)
print(categories)
print(labels)

{'Cat': 0, 'Dog': 1}
['Cat', 'Dog']
[0, 1]


In [3]:
img_size=100
data=[]
target=[]

for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:  
            resized=cv2.resize(img,(img_size,img_size))
            #resizing the image  into 100x100, since we need a fixed common size for all the images in the dataset
            data.append(resized)
            target.append(label_dict[category])
            #appending the image and the label(categorized) into the list (dataset)
        except Exception as e:
            print('Exception:',e)
            #if any exception rasied, the exception will be printed here. And pass to the next image

# Recale and assign catagorical lables

In [4]:
import numpy as np
data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,3))
target=np.array(target)
from keras.utils import np_utils
new_target=np_utils.to_categorical(target)

In [5]:

new_target.shape

(1001, 2)

# CNN Model

In [6]:
data.shape

(1001, 100, 100, 3)

In [7]:
data.shape[1:]

(100, 100, 3)

In [8]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 200)       5600      
                                                                 
 activation (Activation)     (None, 98, 98, 200)       0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 200)      0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 100)       180100    
                                                                 
 activation_1 (Activation)   (None, 47, 47, 100)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 100)      0         
 2D)                                                    

# Splitting data into traning and testing

In [10]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,new_target,test_size=0.1)

In [11]:
train_data.shape

(900, 100, 100, 3)

In [12]:
train_target.shape

(900, 2)

In [13]:
history=model.fit(train_data,train_target,epochs=20,validation_split=0.2)

Epoch 1/20
23/23 [==============================] - 104s 4s/step - loss: 0.7631 - accuracy: 0.5083 - val_loss: 0.6932 - val_accuracy: 0.4778
Epoch 2/20
23/23 [==============================] - 100s 4s/step - loss: 0.6900 - accuracy: 0.5444 - val_loss: 0.7380 - val_accuracy: 0.4167
Epoch 3/20
23/23 [==============================] - 100s 4s/step - loss: 0.6979 - accuracy: 0.5500 - val_loss: 0.6927 - val_accuracy: 0.5111
Epoch 4/20
23/23 [==============================] - 113s 5s/step - loss: 0.6871 - accuracy: 0.5792 - val_loss: 0.6853 - val_accuracy: 0.5500
Epoch 5/20
23/23 [==============================] - 100s 4s/step - loss: 0.6582 - accuracy: 0.6222 - val_loss: 0.7823 - val_accuracy: 0.4056
Epoch 6/20
23/23 [==============================] - 115s 5s/step - loss: 0.6378 - accuracy: 0.6111 - val_loss: 0.6889 - val_accuracy: 0.5056
Epoch 7/20
23/23 [==============================] - 102s 4s/step - loss: 0.5995 - accuracy: 0.6694 - val_loss: 0.6939 - val_accuracy: 0.5833
Epoch 8/20
23

# Saving The Model


In [14]:
model.save('Trained_Model.h5')